In [7]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, RepeatedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

import warnings


warnings.simplefilter("ignore")

## Get data, merget tables, and create new variables

In [47]:
conn = sqlite3.connect("database.sqlite")

In [9]:
# Get data from each table

league = pd.read_sql("select * from league;", conn)

country = pd.read_sql("select * from country;", conn)

team = pd.read_sql("select * from team;", conn)

match = pd.read_sql("select * from match;", conn)

player = pd.read_sql("select * from player;", conn)

plyr_attr = pd.read_sql("select * from player_attributes;", conn)

team_attr = pd.read_sql("select * from team_attributes;", conn) 

In [10]:
# Join league and country tables

league.rename(columns = {"name": "league_name"}, inplace = True)
league = league.merge(country,how="inner",on="id")
league.drop(['id'], axis = 1, inplace = True)

In [11]:
# Select useful columns from match table

match = match[["id","league_id","season","stage","date","match_api_id","home_team_api_id","away_team_api_id",
                 "home_team_goal","away_team_goal","B365H", "B365D","B365A"]]

# Join matches and team name data

match = match.merge(team[["team_api_id","team_long_name"]], how="left", left_on = "home_team_api_id", right_on = "team_api_id")
match.rename(columns = {"team_long_name": "home_team_name"}, inplace = True)
match.drop(["team_api_id"], axis = 1, inplace = True)

match = match.merge(team[["team_api_id","team_long_name"]], how="left", left_on = "away_team_api_id", right_on = "team_api_id")
match.rename(columns = {"team_long_name": "away_team_name"}, inplace = True)
match.drop(["team_api_id"], axis = 1, inplace = True)

In [12]:
# Join matches and league/country data

match = match.merge(league, how="left", left_on = "league_id", right_on = "country_id")
match.drop(["country_id"],axis = 1, inplace = True)
match.rename(columns = {"name": "country"}, inplace = True)

In [13]:
# Tag each match as a home win, draw, or away win

match["home_win"] = np.where(match["home_team_goal"] > match["away_team_goal"], 1, 0)
match["draw"] = np.where(match["home_team_goal"] == match["away_team_goal"], 1, 0)
match["away_win"] = np.where(match["home_team_goal"] < match["away_team_goal"], 1, 0)

match["home_result"] = np.where(match["home_team_goal"] > match["away_team_goal"], "W", 
                                     (np.where(match["home_team_goal"] < match["away_team_goal"], "L", "D")))

# Filter matches to only include England, Spain, and Italy
# Over this time period, these three leagues were the best in the world and also each have 20 teams in them
# This makes for comparisons that make more sense
match = match[match["country"].isin(["England","Spain","Italy"])]

# Convert B365 betting odds to probabilities

match["p_home_win"] = 1 / match["B365H"]
match["p_draw"] = 1 / match["B365D"]
match["p_away_win"] = 1 / match["B365A"]

## Get unique combinations of teams, seasons, stages, and countries

In [14]:
# Get unique combinations of the teams, seasons, stages, and countries before calling get_previous_matches

def create_home_team_pkeys():
    """
    A function to concatenate home team data into a key
    """
    
    team_by_season = match[["home_team_name","home_team_api_id","stage","season","country"]]
    team_by_season["pkey"] = team_by_season.home_team_name.str.cat(team_by_season.home_team_api_id.astype(str), sep='-')
    
    return team_by_season
    
    
def create_away_team_pkeys():
    """
    A function to concatenate away team data into a key
    """
    
    team_by_season = match[["away_team_name","away_team_api_id","stage","season","country"]]
    team_by_season["pkey"] = team_by_season.away_team_name.str.cat(team_by_season.away_team_api_id.astype(str), sep='-')
    
    return team_by_season
    

def concat_stage_season_country(team_by_season):
    """
    A function to combine the home and away keys with stage, season, country
    """
    
    team_by_season["pkey"] = team_by_season.pkey.str.cat(team_by_season.stage.astype(str), sep = "-")
    team_by_season["pkey"] = team_by_season.pkey.str.cat(team_by_season.season.astype(str), sep = "-")
    team_by_season["pkey"] = team_by_season.pkey.str.cat(team_by_season.country.astype(str), sep = "-")
    list_teams_season = list(team_by_season["pkey"].unique())
    
    return list_teams_season


def create_team_tuples(list_teams_season):
    """
    A function to split the combination of team-stage-season-country into parts
    Insert them into a tuple
    Then insert them into a list
    """
    
    team_tups = []

    for i in range(len(list_teams_season)):
        split = list_teams_season[i].split("-", 5)
        team_tups.append((split[0], split[1], split[2], split[3], split[4]))
        
    return team_tups

In [15]:
# Call the functions above to create tuples of the home and away teams
# These tuples will then be submitted as arguments to get_previous_matches

home_pkeys = create_home_team_pkeys()
home_team_list = concat_stage_season_country(home_pkeys)
home_tuples = create_team_tuples(home_team_list)

away_pkeys = create_away_team_pkeys()
away_team_list = concat_stage_season_country(away_pkeys)
away_tuples = create_team_tuples(away_team_list)

## Functions to assist getting previous matches for each game

In [16]:
def get_previous_matches(team_name, team_id, stage, season, country):
    """
    Helper function to get the 10 previous matches played by a team at a given stage of the season
    """
    
    # Get all matches played by a team in a season
    match_team = match[((match["home_team_name"] == team_name) | (match["away_team_name"] == team_name)) &
                        (match["season"] == season)]
    
    # Get the last 10 matches played 
    # This will allow the team's previous run of form to be considered
    match_prev = match_team[match_team["stage"] < stage].sort_values(by = "stage", ascending = False).iloc[0:10,:]
    
    return match_prev

In [17]:
def get_ppg_in_prev_matches(team_name, team_id, stage, season, country):
    """
    A function to get the average points earned per game (not goals) by a team in its previous 10 matches
    """
    
    match_prev = get_previous_matches(team_name, team_id, stage, season, country)
    
    points_last_10 = 0
    
    for index, row in match_prev.iterrows():
        if row["home_win"] == 1 and row["home_team_name"] == team_name:
            points_last_10 += 3
        elif row["away_win"] == 1 and row["away_team_name"] == team_name:
            points_last_10 += 3
        elif row["draw"] == 1:
            points_last_10 += 1
            
    ppg = points_last_10/10
            
    return ppg

In [18]:
def get_goals_in_prev_matches(team_name, team_id, stage, season, country):
    """
    A function to get the average home/away goals for/against by a team in its previous 10 matches
    Returns data as a dictionary
    """
    
    match_prev = get_previous_matches(team_name, team_id, stage, season, country)
    
    # Get the average home and away goals scored in the last 10 matches
    home_goals_for = match_prev.home_team_goal[match_prev.home_team_name == team_name].mean()
    away_goals_for = match_prev.away_team_goal[match_prev.away_team_name == team_name].mean()

    # Get the home and away goals conceded in the last 10 matches
    home_goals_against = match_prev.away_team_goal[match_prev.home_team_name == team_name].mean()
    away_goals_against = match_prev.home_team_goal[match_prev.away_team_name == team_name].mean()
    
    return {"HGF": home_goals_for, "AGF": away_goals_for, "HGA": home_goals_against, "AGA": away_goals_against}

In [19]:
def get_team_form_data(team_name, team_id, stage, season, country):
    """
    Function to get the data referenced in get_goals_in_prev_matches and get_ppg_in_prev_matches
    Returns a dictionary, which will then be put into a list and cast as a dataframe
    """
    
    # Get data on previous goals
    goal_data = get_goals_in_prev_matches(team_name, team_id, stage, season, country)
    
    home_goals_for = goal_data["HGF"]
    home_goals_against = goal_data["AGF"]
    away_goals_for = goal_data["HGA"]
    away_goals_against = goal_data["AGA"]

    # Get the number of points per game earned in the last 10 games (3 for win, 1 for draw)
    prev_points = get_ppg_in_prev_matches(team_name, team_id, stage, season, country)

    # Return data
    return {"team": team_name, "tid": team_id, "country": country,
            "stage": stage, "season": season, "PPG": prev_points, 
            "HGF": home_goals_for, "HGA": home_goals_against, 
            "AGF": away_goals_for, "AGA": away_goals_against}

In [20]:
def get_run_of_form_data(team_tups):
    """
    A function to pull run-of-form data for each team/season/stage combination in the dataset
    """
    run_of_form = []
    
    for t in team_tups:
        team_name = t[0]
        team_id = int(t[1])
        stage = int(t[2])
        season = t[3]
        country = t[4]
        d = get_team_form_data(team_name, team_id, stage, season, country)
        run_of_form.append(d)
                    
    return run_of_form

## Create prediction input dataset

In [21]:
# Get the run of form for each team that played in the dataset
form_home = get_run_of_form_data(home_tuples)
form_away = get_run_of_form_data(away_tuples)

In [22]:
# Convert the list of dictionaries into dataframes
# Filter for only stages after 10, to have a large enough sample when predicting results
# Drop the team name field to avoid duplicates when merging later

form_home = pd.DataFrame(form_home)
form_home = form_home[form_home["stage"] > 10]
form_home = form_home[["tid","country","stage","season","PPG","HGF","HGA","AGF","AGA"]]

form_away = pd.DataFrame(form_away)
form_away = form_away[form_away["stage"] > 10]
form_away = form_away[["tid","country","stage","season","PPG","HGF","HGA","AGF","AGA"]]

In [23]:
# Merge the home dataframe with the full match dataset
match_form = match.merge(form_home, how="left", 
                     left_on = ["home_team_api_id","country","stage","season"], 
                     right_on = ["tid","country","stage","season"])

# Drop tid as it is duplicated
match_form.drop(["tid"], axis = 1, inplace = True)

# Rename columns to refer to home form
match_form.rename(columns = {"PPG": "home_ppg", "HGF": "home_hgf", "HGA":"home_hga",
                        "AGF": "home_agf", "AGA": "home_aga"}, inplace = True)

In [24]:
# Merge the away dataframe with the resulting full+form_home dataset
match_form = match_form.merge(form_away, how="left", 
                     left_on = ["away_team_api_id","country","stage","season"], 
                     right_on = ["tid","country","stage","season"])

# Drop tid as it is duplicated
match_form.drop(["tid"], axis = 1, inplace = True)

# Rename columns to refer to home form
match_form.rename(columns = {"PPG": "away_ppg", "HGF": "away_hgf", "HGA":"away_hga",
                        "AGF": "away_agf", "AGA": "away_aga"}, inplace = True)

# Filter to only include stage > 10
# So that every match has the same amount of data predicting its result
match_form = match_form[match_form["stage"] > 10]

In [25]:
# Create the final dataset for modeling
# Drop the six rows that have NaN values in p_home_win, p_draw, p_away_win
result_df = match_form[["home_ppg", "home_hgf", "home_hga", "home_agf", "home_aga",
                    "away_ppg", "away_hgf", "away_hga", "away_agf", "away_aga",
                    "p_home_win","p_draw","p_away_win", "home_result"]].dropna()

## Model Setup

In [26]:
X = result_df.iloc[:, 0:-1]
y = result_df.iloc[:, -1]

## Logistic Regression: One v Rest

In [27]:
# Logistic Regression with one v rest multiclass parameter

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = LogisticRegression(multi_class = "ovr")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.5398406374501992

In [28]:
# KFold cross validation logistic regression

model = LogisticRegression(multi_class = "ovr")
kfold = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 10, random_state = 0)
log_reg_scores = cross_val_score(model, X, y, cv=kfold)

print(log_reg_scores.mean(), log_reg_scores.std())

0.5415479776007852 0.012728816597034688


In [29]:
# Hyperparameter tuning for one vs rest logistic regression

params = {"C": np.logspace(start = -3, stop = 3, num=50, endpoint = True, base = 10),
          "penalty": ["l1", "l2"]}

log_reg_cv = GridSearchCV(LogisticRegression(multi_class = "ovr"), params, cv=5)
log_reg_cv.fit(X_train, y_train)

print(log_reg_cv.best_params_)

{'C': 0.21209508879201905, 'penalty': 'l2'}


In [30]:
# Best estimator multinomial logistic regression

log_reg_cv.best_estimator_.fit(X_train, y_train)
y_pred = log_reg_cv.best_estimator_.predict(X_test)
print("Accuracy score of best one v rest logistic regression: ", accuracy_score(y_test, y_pred))

Accuracy score of best one v rest logistic regression:  0.5403386454183267


## Logistic Regression: Multinomial Logistic Regression

In [31]:
# Multinomial logistic regression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = LogisticRegression(multi_class="multinomial", solver="lbfgs")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.5493027888446215

In [32]:
# K Fold cross validation multinomial logistic regression

model = LogisticRegression(multi_class="multinomial", solver="lbfgs")
kfold = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 10, random_state = 0)
mlm_scores = cross_val_score(model, X, y, cv=kfold)

print(mlm_scores.mean(), mlm_scores.std())

0.5419215135086897 0.012471660772451343


In [33]:
# Hyperparameter tuning for multinomial logistic regression

params = {"C": np.logspace(start = -3, stop = 3, num=50, endpoint = True, base = 10),
          "penalty": ["l1", "l2"]}

grid_log_reg = GridSearchCV(LogisticRegression(multi_class="multinomial", solver="lbfgs"), params, cv=5)
grid_log_reg.fit(X_train, y_train)

print(grid_log_reg.best_params_)

{'C': 0.15998587196060574, 'penalty': 'l2'}


In [34]:
# Best estimator multinomial logistic regression

grid_log_reg.best_estimator_.fit(X_train, y_train)
y_pred = grid_log_reg.best_estimator_.predict(X_test)
print("Accuracy score of best multinomial logistic regression: ", accuracy_score(y_test, y_pred))

Accuracy score of best multinomial logistic regression:  0.5502988047808764


## Support Vector Classifier

In [35]:
# Support vector classifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = SVC(decision_function_shape = "ovo")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.5293824701195219

In [36]:
# K fold cross validation of SVC

svc = SVC(decision_function_shape = "ovo", kernel='linear', C=1, random_state=42)
kfold = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 10, random_state = 0)
svc_scores = cross_val_score(svc, X, y, cv=kfold)

print(svc_scores.mean(), svc_scores.std())

0.5385286348526426 0.012005485253309809


In [37]:
# Hyperparameter tuning of support vector classifier

params = {"C":[0.01, 0.1, 1],
          "gamma":[0.01, 0.1, 1],
          "kernel": ["linear"]
         }

grid_svc = GridSearchCV(SVC(), params, scoring = "accuracy", n_jobs = -1, verbose = 1, cv = 5)
grid_svc.fit(X_train, y_train)

print(grid_svc.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   10.7s finished


{'C': 1, 'gamma': 0.01, 'kernel': 'linear'}


In [38]:
# Best estimator SVC

grid_svc.best_estimator_.fit(X_train, y_train)
y_pred = grid_svc.predict(X_test)
print("Accuracy score of best support vector classifier: ", accuracy_score(y_test, y_pred))

Accuracy score of best support vector classifier:  0.5313745019920318


## Random Forest Classifier

In [39]:
# Random Forest Classifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = RandomForestClassifier(random_state = 0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.522410358565737

In [40]:
# K fold cross validation Random Forest Classifier

rfc = RandomForestClassifier(random_state=0)
kfold = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 10, random_state = 0)
rfc_scores = cross_val_score(rfc, X, y, cv=kfold)

print(rfc_scores.mean(), rfc_scores.std())

0.5187861588916405 0.015023214202749692


In [41]:
# Hyperparameter tuning of the random forest classifier

params = {"n_estimators": [100, 120, 150], 
          "max_features": ["auto","sqrt","log2"]
         }

grid_rfc = GridSearchCV(RandomForestClassifier(), param_grid = params, scoring = "accuracy", 
                        n_jobs = -1, verbose = 1, cv = 5)

grid_rfc.fit(X_train, y_train)

print(grid_rfc.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   14.0s finished


{'max_features': 'sqrt', 'n_estimators': 100}


In [42]:
# Best estimator random forest classifier

grid_rfc.best_estimator_.fit(X_train, y_train)
y_pred = grid_rfc.best_estimator_.predict(X_test)
print("Accuracy score of best random forest classifier: ", accuracy_score(y_test, y_pred))

Accuracy score of best random forest classifier:  0.524402390438247


## K Neighbors Classifier

In [43]:
# K Neighbors classifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = KNeighborsClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.43227091633466136

In [44]:
# K fold cross validation K Neighbors classifier

knc = KNeighborsClassifier()
kfold = RepeatedStratifiedKFold(n_splits = 10, random_state=0)
knc_scores = cross_val_score(knc, X, y, cv=kfold)

print(knc_scores.mean(), knc_scores.std())

0.42742533966936613 0.016652801384845817


In [45]:
# Hyperparameter tuning of the K Neighbors classifier

params = {"n_neighbors": [3, 4, 5, 6, 7, 8], 
          "weights": ["uniform","distance"],
          "algorithm": ["auto", "ball_tree","kd_tree", "brute"],
          "p":[2]
         }

grid_knc = GridSearchCV(KNeighborsClassifier(), param_grid = params, scoring = "accuracy", 
                        n_jobs = -1, verbose = 1, cv = 5)

grid_knc.fit(X_train, y_train)

print(grid_knc.best_params_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    2.4s


{'algorithm': 'auto', 'n_neighbors': 8, 'p': 2, 'weights': 'distance'}


[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:    8.2s finished


In [46]:
# Best estimator K Neighbors classifier

grid_knc.best_estimator_.fit(X_train, y_train)
y_pred = grid_knc.best_estimator_.predict(X_test)
print("Accuracy score of best K Neighbors Classifier: ", accuracy_score(y_test, y_pred))

Accuracy score of best K Neighbors Classifier:  0.4716135458167331
